## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откроем файл с данными и изучим общую информацию. 

In [1]:
# Импорт и первичный анализ
import pandas as pd

df = pd.read_csv('/datasets/data.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод

Первично по данным можно увидеть:
1. days_employed, total_income - в столбцах есть пропуски. Возможная причина - невнесение данных клиентами или сотрудниками в анкету.
2. Типы данных по столбцам правильные, там где предполагаются числа - числовые, там где строки - object.
3. education, разный регистр записи.
4. days_employed - имеет отрицательные значения, но общий трудовой стаж не может быть отрицательным. Также значения у пенсионеров положительне, шестизначные, приведение в формате дни/365 не даст нужной информации. Нет возможности узнать, что имеется ввиду.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
# days_employed просто заполним средним.
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].mean())

In [4]:
# total_income используется для прогноза в дальнейшем, поэтому заполним более точно.
# В зависимости от income_type установим значения в total_income.

income_type = df['income_type'].unique()
for i in income_type:
    df.loc[(df['total_income'].isna()) & (df['income_type'] == i), 'total_income'] = df[df['income_type'] == i]['total_income'].median()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

1. days_employed заполняем средним.
2. total_income медианой с учетом income_type, при расчете дохода это более правильно.
3. Пропущенных значений больше нет.

### Замена типа данных

In [6]:
# Замена days_employed на целочисленный тип и изменение на положительное значение.
df['days_employed'] = df['days_employed'].astype(int)
df.loc[df['days_employed'] < 0, 'days_employed'] = df[df['days_employed'] < 0]['days_employed'] * -1

In [7]:
# Замена total_income на целочисленный тип.
df['total_income'] = df['total_income'].astype(int)

In [8]:
# Замена отрицательного значения детей. Возможно ошибка как с отрицательным значением стажа, поэтому умножим на -1.
df.loc[df['children'] < 0, 'children'] = df[df['children'] < 0]['children'] * -1
# Есть значение в 20 детей и не единожды, скорее всего опечатка в 0. Заменим на 2.
df.loc[df['children'] == 20, 'children'] = 2
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

1. Так как числовые данные в df уже были в цифровом формате, то можем применить метод astype()

### Обработка дубликатов

In [9]:
# Посмотрим, что вообще из себя представляют дубликаты.
df[df.duplicated(keep=False)].sort_values(by='purpose').head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
13639,0,63046,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,автомобиль
8178,0,63046,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,автомобиль
120,0,63046,46,среднее,1,женат / замужем,0,F,сотрудник,0,142594,высшее образование
16378,0,63046,46,среднее,1,женат / замужем,0,F,сотрудник,0,142594,высшее образование
20702,0,63046,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование
8583,0,63046,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
12389,0,63046,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование
1511,0,63046,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
3609,0,63046,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,жилье
18428,0,63046,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,жилье


In [10]:
# Для эффективной обработки дубликатов, нужно привести данные к единообразному виду.
df['education'] = df['education'].str.lower()
df.duplicated().sum()

71

In [11]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

### Вывод

1. 71 дубликат удален, использован стандартный метод, так как значения в столбцах приведены к единообразному виду. Мы видим, что часть дубликатов появилась в результате заполнения отсутствующих значений. Возможно, было бы меньше дубликатов, если бы не было пропусков.

### Лемматизация

In [12]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

text = []

# Так как lemmatize работает со строкой, сделаем одну большую строку из значений столбца и передадим её в библиотеку.
for i in range(len(df['purpose'])):
    text.append(df['purpose'][i])
    
lemmas = m.lemmatize(str(text))
x = Counter(lemmas)
x.most_common()

[(' ', 33570),
 ("', '", 21453),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('подержать', 853),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('приобретение', 461),
 ('профильный', 436),
 ('подержанный', 111),
 ("['", 1),
 ("']\n", 1)]

### Вывод

1. По леммам видно, что со значительным отрывом лидирует недвижимость.
2. Также видно, что кредит берётся на покупки, а не на другие цели, вроде лечения, свадьбы.

### Категоризация данных

In [13]:
# Имеет смысл категорировать purpose

df['purpose'].unique()

# Видим, что целей в purpose больше чем лемм тематически. Чтобы не перебирать руками совместим лемматизацию и категоризацию.
# Плюс цель иногда написана с ошибкой, простой поиск if не будет эффективным.
# По леммам можно выделить следующие цели: 
# Недвижимость
# Автомобиль
# Образование
# Свадьба
# Бытовые, если будут другие цели

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [14]:
def credit_target(row):    
    lemmas = m.lemmatize(row)
    if 'недвижимость' in lemmas:
        return 'недвижимость'
    elif 'жилье' in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    else:
        return 'бытовые'


df['credit_target'] = df['purpose'].apply(credit_target)
df['credit_target'].unique()
# Других целей не оказалось

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

In [15]:
df['credit_target'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: credit_target, dtype: int64

In [16]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,credit_target
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


### Вывод

1. При категоризации целей кредитования мы выявили, что подавляющее большинство число сделок приходится на недвижимость.
2. Видим что цели кредитования стандртны, как правило, это крупные затраты.

### Шаг 3. Ответьте на вопросы

#### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [17]:
# Посмотрим, как визуально обстоят дела с возвратом.

df.groupby('children')['debt'].value_counts()

# Если детей 5, то 100% возврат.

children  debt
0         0       13028
          1        1063
1         0        4410
          1         445
2         0        1926
          1         202
3         0         303
          1          27
4         0          37
          1           4
5         0           9
Name: debt, dtype: int64

In [18]:
# Выведем в красивом виде.

children = df['children'].unique()
children.sort(axis=0)

for i in children:
    val = df[df['children'] == i]['debt'].value_counts()
    
    try:
        perc = val[1] / val[0]
    except:
        perc = 0
    
    if i == 0:
        i = 'нет'
        
    print(f'Если у заёмщика {i} детей, то процент невозврата - {perc :.2%}')
    print()

Если у заёмщика нет детей, то процент невозврата - 8.16%

Если у заёмщика 1 детей, то процент невозврата - 10.09%

Если у заёмщика 2 детей, то процент невозврата - 10.49%

Если у заёмщика 3 детей, то процент невозврата - 8.91%

Если у заёмщика 4 детей, то процент невозврата - 10.81%

Если у заёмщика 5 детей, то процент невозврата - 0.00%



### Вывод

1. Интересно, что максимальный процент возврата кредита у заёмщиков, у которых нет детей.
2. Максимальный процент невозврата у семеё с 4 детьми.
3. Семей с 5 детьми не много, поэтому нельзя сказать, что высока вероятность исполнения обязательств у семей с 5 детьми.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [19]:
# Посмотрим, как визуально обстоят дела с возвратом.

df.groupby('family_status')['debt'].value_counts()

family_status          debt
Не женат / не замужем  0        2536
                       1         274
в разводе              0        1110
                       1          85
вдовец / вдова         0         896
                       1          63
гражданский брак       0        3763
                       1         388
женат / замужем        0       11408
                       1         931
Name: debt, dtype: int64

In [20]:
# Выведем в красивом виде. Сделаем функцию, так как вывод будет повторяться.

def print_result(column, phrase):
    values = df[column].unique()

    for i in values:
        val = df[df[column] == i]['debt'].value_counts()

        try:
            perc = val[1] / val[0]
        except:
            perc = 0

        print(f'Если {phrase} {i}, то процент невозврата - {perc :.2%}')
        print()

In [21]:
print_result('family_status', 'семейное положение')

Если семейное положение женат / замужем, то процент невозврата - 8.16%

Если семейное положение гражданский брак, то процент невозврата - 10.31%

Если семейное положение вдовец / вдова, то процент невозврата - 7.03%

Если семейное положение в разводе, то процент невозврата - 7.66%

Если семейное положение Не женат / не замужем, то процент невозврата - 10.80%



### Вывод

1. Максимальный процент возврата у вдовствующих.
2. Максимальный процент невозврата тех кто не женат/не замужем.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [22]:
# Разделим заемщиков по квантилям. А именно:
# бедные
# ниже среднего
# выше среднего
# богатые

df['total_income'].quantile([0.25,0.5,0.75])

0.25    107623.00
0.50    142594.00
0.75    195820.25
Name: total_income, dtype: float64

In [23]:
def fin_status(row):
    quantile = df['total_income'].quantile([0.25,0.5,0.75])
    
    if row <= quantile[0.25]:
        return 'бедные'
    elif quantile[0.25] < row <= quantile[0.5]:
        return 'ниже среднего'
    elif quantile[0.5] < row <= quantile[0.75]:
        return 'выше среднего'
    elif row > quantile[0.75]:
        return 'богатые'

df['fin_status'] = df['total_income'].apply(fin_status)
df.groupby('fin_status')['debt'].value_counts()

fin_status     debt
бедные         0       4937
               1        427
богатые        0       4981
               1        383
выше среднего  0       4799
               1        448
ниже среднего  0       4996
               1        483
Name: debt, dtype: int64

In [24]:
# Выведем.

print_result('fin_status', 'финансовое положение')

Если финансовое положение богатые, то процент невозврата - 7.69%

Если финансовое положение ниже среднего, то процент невозврата - 9.67%

Если финансовое положение выше среднего, то процент невозврата - 9.34%

Если финансовое положение бедные, то процент невозврата - 8.65%



### Вывод

1. Богатые возвращают кредит чаще всего.
2. Невозврат кредита больше всего у тех, у кого финансовое положение ниже среднего.

#### Как разные цели кредита влияют на его возврат в срок?

In [25]:
# Выведем.
# Мы ранее добавили столбец с целями кредитования, его и используем. 

print_result('credit_target', 'цель кредитования')

Если цель кредитования недвижимость, то процент невозврата - 7.80%

Если цель кредитования автомобиль, то процент невозврата - 10.33%

Если цель кредитования образование, то процент невозврата - 10.16%

Если цель кредитования свадьба, то процент невозврата - 8.70%



### Вывод

1. Меньше всего невозврата в кредитах на недвижимость.
2. Больше всего невозврата в кредитах на автомобиль.

### Шаг 4. Общий вывод

Таким образом можно выявить аватар наиболее и наименее благонадежного заемщика.

Идеальный заёмщик - это человек, у которого нет детей, он вдовствующий, богатый и берёт кредит на недвижимость.
Следует применить технологии хеджирования рисков, если заёмщик - это человек, у которого 4 детей, не женат/не замужем, доход ниже среднего и цель автомобиль.

В принципе, такие выводы кажутся весьма логичными.